### __Importing the dependencies__

In [49]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### __Data Collection__

In [50]:
# Loading the dataset
movies_data = pd.read_csv('./data/movies.csv')

In [51]:
# Printing the first 5 rows of the dataframe
movies_data.head(5)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [52]:
# Number of rows and columns in the dataframe
movies_data.shape

(4803, 24)

### __Data Preprocessing__

In [53]:
# Selecting the relevant features, which will be used for content-based filtering
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

In [54]:
# Replacing the null values with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [55]:
# Combining all the 5 selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

In [56]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [57]:
# Converting the text data to future vectors
vectorizer = TfidfVectorizer()

In [58]:
# Creating the feature vectors
feature_vectors = vectorizer.fit_transform(combined_features)

In [59]:
# Printing the feature vectors
print(feature_vectors)

  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964
  (4801, 17266)	0.28860981849329476
  (4801, 13835)	0.27870029291200094
  (4801, 13175)	0.28860981849329476
  (4801, 171

### __Cosine Similarity__

In [60]:
# Getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [61]:
# Printing the similarity scores
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [62]:
# Printing the shape of the similarity matrix
print(similarity.shape)

(4803, 4803)


### __Getting the movie name from the user__

In [63]:
# Creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()

In [64]:
# Printing the list of all movie titles
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [65]:
# Getting the movie name from the user
movie_name = input('Enter your favourite movie name: ')

In [66]:
# Finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

In [67]:
# Printing the list of all close matches
print(find_close_match)

['Prom', 'Airborne', 'Nixon']


In [68]:
# Finding the most close match for the movie name given by the user
close_match = find_close_match[0]

In [69]:
# Printing the most close match
print(close_match)

Prom


In [70]:
# Finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

In [71]:
# Printing the index of the movie with title
print(index_of_the_movie)

3266


In [72]:
# Getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [73]:
# Printing the similarity score
print(similarity_score)

[(0, 0.006832418966817807), (1, 0.022335036866533514), (2, 0.012201004434031392), (3, 0.0), (4, 0.03644022720995055), (5, 0.03854917356129364), (6, 0.0), (7, 0.0), (8, 0.0495161500675359), (9, 0.0), (10, 0.0), (11, 0.0), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.028701536843712135), (19, 0.005925066646369554), (20, 0.033201162046182925), (21, 0.0), (22, 0.006170787824220303), (23, 0.04475740848557152), (24, 0.023856088136103407), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.07042556976667233), (31, 0.0), (32, 0.00877432525589754), (33, 0.0), (34, 0.04169379443506636), (35, 0.0), (36, 0.0), (37, 0.009971974310066588), (38, 0.02705246719515969), (39, 0.0), (40, 0.007628135665523365), (41, 0.068196839219273), (42, 0.0035969334510623082), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.02312779735048387), (48, 0.028338384538724313), (49, 0.0057174591860955125), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.02771314774196004), (54, 0.0), (55, 0.0

In [74]:
# Printing the number of similar movies
print(len(similarity_score))

4803


In [75]:
# Sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

In [76]:
# Printing the most similar movies
print(sorted_similar_movies)

[(3266, 1.0), (3915, 0.188008343982409), (3218, 0.18328888243785807), (2810, 0.17636208081164284), (1184, 0.17503331516786938), (3049, 0.16611292035165584), (2477, 0.16093164618510927), (3253, 0.16065306641638974), (1119, 0.1593305770817414), (4157, 0.1585780915628437), (3187, 0.15215484222509296), (3731, 0.14665613842496103), (3747, 0.14633989569099076), (1335, 0.14197471094679098), (1375, 0.14164740247659025), (3807, 0.14097255855180468), (2565, 0.13466384171723625), (1906, 0.13454382778506985), (904, 0.13414594931237278), (2132, 0.1281062578377471), (2135, 0.1250856212443977), (3044, 0.1223164630066387), (2330, 0.12066921414475792), (1756, 0.11879582183399737), (4545, 0.11680619427586343), (915, 0.11563785129346545), (914, 0.11540424823345907), (4723, 0.114653534784481), (4463, 0.11350403582331273), (2333, 0.11305474771703242), (2718, 0.11253087429002291), (3263, 0.11226141528376704), (3919, 0.11171207039814132), (2572, 0.10915110133674538), (2283, 0.10771351748066427), (2392, 0.107

In [77]:
# Printing the name of similar movies based on the index
print('Movies suggested for you\n')

for i, movie in enumerate(sorted_similar_movies):
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i+1, '.',title_from_index)

Movies suggested for you

1 . Prom
2 . Trippin'
3 . The DUFF
4 . American Pie
5 . 17 Again
6 . Sleepover
7 . Jennifer's Body
8 . Drive Me Crazy
9 . 21 Jump Street
10 . The Curse of Downers Grove
11 . Hamlet 2
12 . High School Musical
13 . Trapeze
14 . How to Be Single
15 . Sky High
16 . The Loved Ones
17 . Grosse Pointe Blank
18 . The Girl Next Door
19 . As Good as It Gets
20 . Vampires Suck
21 . Doubt
22 . Get Over It
23 . Mean Girls
24 . About a Boy
25 . Detention of the Dead
26 . Stepmom
27 . Central Intelligence
28 . The Gallows
29 . Not Cool
30 . Peggy Sue Got Married


### __Movie Recommendation System__

In [78]:
# User input for movie name
movie_name = input('Enter your favourite movie name: ')

In [79]:
list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

In [80]:
print('Movies suggested for you\n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you

1 . Rio
2 . Rio 2
3 . The Elephant Man
4 . The Devil Wears Prada
5 . Pitch Black
6 . Ice Age: Dawn of the Dinosaurs
7 . Brokeback Mountain
8 . Chappie
9 . Conquest of the Planet of the Apes
10 . Turbulence
11 . George of the Jungle
12 . Wreck-It Ralph
13 . Ice Age: The Meltdown
14 . V for Vendetta
15 . Dreamcatcher
16 . The Land Before Time
17 . Exotica
18 . Without a Paddle
19 . This Is 40
20 . Miss Potter
21 . My Name Is Khan
22 . The Three Stooges
23 . The Hunting Party
24 . The Other Side of Heaven
25 . How to Train Your Dragon
26 . Rachel Getting Married
27 . Alvin and the Chipmunks
28 . Stuart Little 2
29 . The Iron Giant
